In [ ]:
import getpass

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
PROJECT = "<project>"

In [ ]:
! git clone https://github.com/INCF/neuroshapes.git

## Initialize

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
config = {
    "Model": {
		"name": "RdfModel",
		"origin": "directory",
		"source": "./neuroshapes/shapes/",
		"context": {
            # FIXME
			"iri": "<FIXME>",
		},
	},
	"Store": {
        "name": "BlueBrainNexus",
		"endpoint": "https://bbp.epfl.ch/nexus/v1",
        "bucket": f"bbp/{PROJECT}",
		"token": TOKEN,
	},
}

In [ ]:
forge = KnowledgeGraphForge(config)

## Search

In [ ]:
forge.types()

In [ ]:
p = forge.paths("Trace")

In [ ]:
data = forge.search(p.type == "Trace")

In [ ]:
len(data)

In [ ]:
DISPLAY_LIMIT = 10

In [ ]:
forge.as_dataframe(data[:DISPLAY_LIMIT])

## Download

In [ ]:
files = forge.reshape(data, keep=["distribution.name", "distribution.contentUrl"])

In [ ]:
forge.as_dataframe(files)

In [ ]:
dirpath = "./downloaded/"

In [ ]:
forge.download(data, "distribution.contentUrl", dirpath)